In [1]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertConfig, Trainer, TrainingArguments
import torch
from scipy.stats import pearsonr
import numpy as np
from transformers.models.bert.modeling_bert import BertModel
from torch import nn
from transformers.trainer_utils import set_seed
import random
set_seed(42)

2024-06-08 23:59:07.256100: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-08 23:59:07.256238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-08 23:59:07.367696: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!wget -O train.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv"
!wget -O trial.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv"

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


--2024-06-09 00:00:38--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1746979 (1.7M) [text/plain]
Saving to: 'train.tsv'

train.tsv           100%[===================>]   1.67M  --.-KB/s    in 0.04s   

2024-06-09 00:00:39 (45.1 MB/s) - 'train.tsv' saved [1746979/1746979]

--2024-06-09 00:00:40--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connec

In [75]:
# Load data
train = pd.read_table("/kaggle/working/train.tsv")
trial = pd.read_table("/kaggle/working/trial.tsv")

# Split data for training and evaluation
train_data = train[:-100]
eval_data = trial[:35].reset_index(drop=True)

def data_augmentation(df):
    augmented_sentences = []
    augmented_tokens = []
    augmented_complexity = []
    for _, row in df.iterrows():
        sentence = row['sentence']
        token = row['token']
        complexity = row['complexity']
        words = sentence.split()
        if len(words) > 1:
            # Random Insertion
            if random.random() < 0.2:
                words.insert(random.randint(0, len(words) - 1), random.choice(words))

            # Random Swap
            if random.random() < 0.2:
                i, j = random.sample(range(len(words)), 2)
                words[i], words[j] = words[j], words[i]
            # Random Deletion
            if random.random() < 0.2:
                del words[random.randint(0, len(words) - 1)]

        augmented_sentences.append(' '.join(words))
        augmented_tokens.append(token)
        augmented_complexity.append(complexity)

    augmented_df = pd.DataFrame({'sentence': augmented_sentences, 'token': augmented_tokens, 'complexity': augmented_complexity})
    return pd.concat([df, augmented_df], ignore_index=True)

# Apply data augmentation to train_data
train_data = data_augmentation(train_data)

In [76]:
train_data.drop(["id", "corpus"], axis = 1, inplace = True)

In [77]:
train_data

,sentence,token,complexity
0,"Behold, there came up out of the river seven c...",river,0.000000
1,I am a fellow bondservant with you and with yo...,brothers,0.000000
2,"The man, the lord of the land, said to us, 'By...",brothers,0.050000
3,Shimei had sixteen sons and six daughters; but...,brothers,0.150000
4,He has put my brothers far from me.\tbrothers\...,sons,0.055556
...,...,...,...
14259,The transatlantic partnership ceased to be mer...,transatlantic,0.470588
14260,"– Mr President, ladies and gentlemen, I take t...",trepidation,0.470588
14261,"ERDF, ESF and Cohesion Fund: provisions relati...",Cohesion,0.470588
14262,General provisions on the European Regional De...,Cohesion,0.500000


In [78]:
# Create datasets from pandas DataFrames
train_dataset = Dataset.from_pandas(train_data)
eval_dataset = Dataset.from_pandas(eval_data)

In [81]:
# Preprocessing function
def remove_quotations(text):
    text = text.replace('"', '')
    text = text.replace("'", "")
    return text

def preprocess_function(examples):
    sentences = [remove_quotations(f"[CLS] {s} [SEP] {t}") for s, t in zip(examples['sentence'], examples['token'])] 
    tokenized_inputs = tokenizer(sentences, padding=True, truncation=True, max_length=64)
    tokenized_inputs['labels'] = examples['complexity']
    return tokenized_inputs

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [82]:
# Apply preprocessing to datasets
train_dataset = train_dataset.map(preprocess_function, batched=True, batch_size=64)
eval_dataset = eval_dataset.map(preprocess_function, batched=True, batch_size=16)

Map:   0%|          | 0/14264 [00:00<?, ? examples/s]

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [117]:
model_path = "prajjwal1/bert-medium"
from transformers import AutoTokenizer, AutoModelForMaskedLM

config = BertConfig.from_pretrained(model_path, num_labels=1, dropout=0.1, attention_dropout=0.1)

model = AutoModelForSequenceClassification.from_pretrained(model_path, config = config)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-medium and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [118]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 512, padding_idx=0)
      (position_embeddings): Embedding(512, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-7): 8 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-1

In [125]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/model',
    report_to="none",
    do_train=True,
    do_eval=True,
    num_train_epochs=3,
    warmup_steps=100,
    weight_decay=0.01,
    logging_strategy='steps',
    logging_dir='./logs',
    logging_steps=20,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    lr_scheduler_type='linear', 
    learning_rate=4e-5,
)

# Compute metrics function
def compute_metrics(pred):
    preds = np.squeeze(pred.predictions)
    return {"Pearson": pearsonr(preds, pred.label_ids)[0]}


# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [120]:
from torch import cuda
import torch
device = 'cuda' if cuda.is_available() else 'cpu'

In [121]:
trainer.train()

Step,Training Loss,Validation Loss,Pearson
50,0.034400,0.045976,0.191736
100,0.028200,0.044446,0.262092
150,0.022100,0.005429,0.493086
200,0.023800,0.039771,0.175657
250,0.022200,0.031562,0.292797
300,0.017800,0.039137,0.355760
350,0.029300,0.006736,0.302324
400,0.023900,0.012337,0.271580
450,0.014700,0.020559,0.246089
500,0.016700,0.007242,0.101000


TrainOutput(global_step=5349, training_loss=0.008452752126273897, metrics={'train_runtime': 254.4202, 'train_samples_per_second': 168.194, 'train_steps_per_second': 21.024, 'total_flos': 418744272337920.0, 'train_loss': 0.008452752126273897, 'epoch': 3.0})